<a href="https://colab.research.google.com/github/JayRGopal/CRAFT-pytorch/blob/master/Glaucoma_Manuscript_Gopal_Open_Source.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import glob
from scipy.misc import *

In [ ]:
from skimage.util import view_as_blocks, view_as_windows, montage
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import randint
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, transforms
import time
import os
import copy
import torch
from torch import tensor 
from torch.autograd import Variable
from torch import randn, matmul
import matplotlib.pyplot as plt
from progressbar import progressbar
from google.colab import files
import matplotlib.pyplot as plt
import time
from torchvision.models import *
from PIL import Image

In [ ]:
def plot(x):
    fig, ax = plt.subplots()
    im = ax.imshow(x)
    ax.axis('off')
    fig.set_size_inches(8, 8)
    plt.show()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

In [ ]:
def read_ims(directory, imsz):


    main_dir = os.getcwd()
    os.chdir(directory)

    num_channels = 3  # remove

    num_ims = sum([len(files) for (r, d, files) in os.walk(directory)])


    imgs = np.zeros([num_ims, imsz, imsz, num_channels])

    im_num = 0
    class_num = 0



    for filename in sorted(os.listdir(os.getcwd())):
        print(filename)
        im = Image.open(filename)
        im = np.array(im.resize((imsz, imsz)))


        imgs[im_num, :, :, :] = im



        im_num += 1
    os.chdir(directory)
    os.chdir(main_dir)
    return (imgs)


In [ ]:
!pwd

In [ ]:
path = '/content/drive/My Drive/OpticsAI/GlaucomaRetinalData/Images'

In [ ]:
imsz = 227

In [ ]:
X = read_ims(path, imsz)

In [ ]:
X = X /255

In [ ]:
Y = np.zeros((X.shape[0],4))

In [ ]:
import csv
index = 0  
with open('/content/drive/My Drive/OpticsAI/GlaucomaRetinalData/GlaucomaDataSuspectCSV.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            line_count += 1
            if(index < Y.shape[0]):
              Y[index, int(row[1])] = 1
            index+=1
    print(f'Processed {line_count} lines.')

In [ ]:
r = np.random.permutation(X.shape[0])
X = X[r,:,:,:]
Y = Y[r,:]

In [ ]:
X_train = X[0:int(0.8*X.shape[0])]
X_val = X[int(0.8*X.shape[0]):X.shape[0]] 

Y_train = Y[0:int(0.8*Y.shape[0])]
Y_val = Y[int(0.8*Y.shape[0]):Y.shape[0]] 

In [ ]:
Data = {'train':X_train,'val':X_val}
Labels = {'train':Y_train,'val':Y_val}

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation([-15, 15], resample=False, expand=False, center=None),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

image_datasets = {x: torch.utils.data.TensorDataset(torch.tensor(Data[x], dtype=torch.float).transpose(3,1),torch.tensor(Labels[x], dtype=torch.float)) for x in ['train', 'val']}
dataloaders    = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=4) for x in ['train', 'val']}
dataset_sizes  = {x: len(image_datasets[x]) for x in ['train', 'val']}

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, torch.argmax(labels,1))

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == torch.argmax(labels.data,1))

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model


In [ ]:
net = resnext101_32x8d(pretrained=True).to(device)

In [ ]:
optimizer_ft = optim.SGD(net.parameters(), lr=0.003162, momentum=0.75)
criterion = nn.CrossEntropyLoss()
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.9)

In [ ]:
net = train_model(net, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=100)